In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso, Ridge

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from tensorflow.keras.layers import Dropout




In [ ]:

# Load the training and test data
df_train_orig = pd.read_csv('train.csv')
df_test_orig = pd.read_csv('test.csv')


In [ ]:

# Make a copy of the original data
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()
target_col = 'price_doc'
row_id_col = 'row ID'

# Label encode the "sub_area" column
label_encoder = LabelEncoder()
df_train['sub_area'] = label_encoder.fit_transform(df_train['sub_area'])
df_test['sub_area'] = label_encoder.transform(df_test['sub_area'])

# Create dummy variables for categorical features
X_train_full = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
y = df_train[target_col]

X_test = pd.get_dummies(df_test, drop_first=True)

# Extract the 'row ID' column for later use and remove it from X_test
row_ids = df_test['row ID']
X_test.drop(['row ID'], axis=1, inplace=True)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y, test_size=0.1, random_state=42)


In [ ]:
print('X_train shape:', X_train.shape)
print('X_val shape:', X_val.shape)
print('X_test shape:', X_test.shape)

In [ ]:

# Step 2: Use a Decision Tree Regressor to get the 100 most important features
tree_regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
tree_regressor.fit(X_train, y_train)
importances = tree_regressor.feature_importances_


In [ ]:

# Get indices of the top 100 features
top_100_feature_indices = np.argsort(importances)[-20:]

# Select the top 100 features
X_train = X_train.iloc[:, top_100_feature_indices]
X_val = X_val.iloc[:, top_100_feature_indices]
X_test = X_test.iloc[:, top_100_feature_indices]


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)



In [ ]:
print('X_train_scaled shape:', X_train.shape)
print('X_val_scaled shape:', X_val.shape)
print('X_test_scaled shape:', X_test.shape)

In [ ]:
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)
X_test = pca.transform(X_test)

In [ ]:
threshold = 0.1

selector = VarianceThreshold(threshold=threshold)

X_train = selector.fit_transform(X_train)
X_val = selector.transform(X_val)
X_test = selector.transform(X_test)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
print('X_train_pca shape:', X_train.shape)
print('X_val_pca shape:', X_val.shape)
print('X_test_pca shape:', X_test.shape)


In [ ]:
# Step 3: Use Forward Selection to get the first 10 best features
selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=10)
selector.fit(X_train, y_train)

# Get the selected feature indices
selected_feature_indices = selector.get_support()


In [ ]:

# Select the first 10 best features
X_train = X_train.iloc[:, selected_feature_indices]
X_val = X_val.iloc[:, selected_feature_indices]
X_test = X_test.iloc[:, selected_feature_indices]


In [ ]:
print('X_train_top10 shape:', X_train.shape)
print('X_val_top10 shape:', X_val.shape)
print('X_test_top10 shape:', X_test.shape)


In [16]:

# Step 4: Use Polynomial Features with interaction on the selected features
poly = PolynomialFeatures(degree=2, interaction_only=False)
X_train = poly.fit_transform(X_train)
X_val = poly.transform(X_val)
X_test = poly.transform(X_test)

print('X_train_poly shape:', X_train.shape)
print('X_val_poly shape:', X_val.shape)
print('X_test_poly shape:', X_test.shape)


X_train_poly shape: (163356, 231)
X_val_poly shape: (18151, 231)
X_test_poly shape: (77789, 231)


In [17]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.neural_network import MLPRegressor

# Step 5: Train and fit a Linear Regression model on the polynomial feature set
model = Lasso()
model = MLPRegressor(
    hidden_layer_sizes=(256, 128,128, 64, 32, 16, 8,),
    activation='relu',
    solver='adam',
    random_state=42,
    verbose=True,
    max_iter=10,
    early_stopping=True,
    batch_size=64,
    alpha=1e-5,  # L2 regularization term
)

# Evaluate the model on the validation set
# predictions = model.predict(X_val_poly)
# rmse = np.sqrt(mean_squared_error(y_val, predictions))


model.fit(X_train, y_train)


# # Evaluate the model on the validation set
predictions = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, predictions))

print(f"RMSE on validation set:", rmse)

# Make predictions on the test data
test_predictions = model.predict(X_test)

# Create a DataFrame with 'row ID' and predictions

result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

# Save the results to a CSV file


result_df.to_csv('predictions_40.csv', index=False)




Iteration 1, loss = 122884863367736.71875000
Validation score: 0.580738
Iteration 2, loss = 96596866959366.59375000
Validation score: 0.607051
Iteration 3, loss = 93561130104557.50000000
Validation score: 0.613973
Iteration 4, loss = 92398812851247.82812500
Validation score: 0.615424
Iteration 5, loss = 91286341146669.68750000
Validation score: 0.617856
Iteration 6, loss = 90506285552266.21875000
Validation score: 0.623605
Iteration 7, loss = 89833139821339.18750000
Validation score: 0.623902
Iteration 8, loss = 89191465247764.76562500
Validation score: 0.625761
Iteration 9, loss = 88493171729815.96875000
Validation score: 0.626658
Iteration 10, loss = 87868281903659.96875000
Validation score: 0.606138
RMSE on validation set: 13384352.660707898


/Users/yahyaahmedkhan/miniconda3/envs/tfenv2/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

# Create a custom Adam optimizer with a reduced learning rate
custom_optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate as needed

model = keras.Sequential([
    keras.layers.Input(shape=(231,)),  # Input layer with 274 input features

    Dropout(0.4),  # Add a Dropout layer with a dropout rate of 0.4 (40% dropout rate)
    keras.layers.Dense(64, activation='relu'),  # Another hidden layer with 64 neurons and ReLU activation
    Dropout(0.3),  # Add a Dropout layer with a dropout rate of 0.3 (30% dropout rate)
    keras.layers.Dense(32, activation='relu'),  # Yet another hidden layer with 32 neurons and ReLU activation
    Dropout(0.2),  # Add a Dropout layer with a dropout rate of 0.2 (20% dropout rate)
    keras.layers.Dense(1)  # Output layer for regression, no activation function
])

model.compile(optimizer=custom_optimizer,
              loss='mean_squared_error',  # Use mean squared error (MSE) for regression
              metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mae'])  # RMSE as a custom metric


In [20]:
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
2553/2553 [==============================] - 2s 816us/step - loss: 571229912170496.0000 - root_mean_squared_error: 23900416.0000 - mae: 13298056.0000
Epoch 2/10
2553/2553 [==============================] - 2s 814us/step - loss: 283705306251264.0000 - root_mean_squared_error: 16843554.0000 - mae: 9616452.0000
Epoch 3/10
2553/2553 [==============================] - 2s 802us/step - loss: 268947563741184.0000 - root_mean_squared_error: 16399621.0000 - mae: 9236188.0000
Epoch 4/10
2553/2553 [==============================] - 3s 1ms/step - loss: 263286746513408.0000 - root_mean_squared_error: 16226113.0000 - mae: 9056791.0000
Epoch 5/10
2553/2553 [==============================] - 2s 861us/step - loss: 260447135596544.0000 - root_mean_squared_error: 16138375.0000 - mae: 8894540.0000
Epoch 6/10
2553/2553 [==============================] - 2s 800us/step - loss: 256991029100544.0000 - root_mean_squared_error: 16030940.0000 - mae: 8752224.0000
Epoch 7/10
2553/2553 [===================

In [ ]:
import sys

import tensorflow as tf
import tensorflow.keras
import pandas as pd
import sklearn as sk
import scipy as sp
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
